In [247]:
# Importing necessary helpful libiraries
import numpy as np
import pandas as pd

# File Reading and Setup

### Instructions 
Setting up the file, put string of raw data into csv

## Simpily Go to Cell,  Run all Cells 
Ex. 

rawData = pd.read_csv('dummy.csv')

In [242]:
rawData = pd.read_csv("rdt.csv", sep = ',')
#Make sure that data is imported with the correct format 
#If the file is not corretcly separated by semicoln, the method will not run and result in error, indicating possible worng \n
#format of input file
rawData.dropna(how = 'all')
#Check if each column is in correct order 
dilutionTable = pd.read_csv("dilutionTable.csv", sep = ',')
blank_table = pd.DataFrame({"Sample ID"   : rawData['Sample ID'],
                           "AnalyteName" : rawData['Analyte Name'],
                           "Conc (Calib)": rawData['Conc (Calib)'],
                           "SD (Calib)" : rawData['SD (Calib)'],
                          })
blank_table = blank_table.fillna(value = 0.0)
blank_table = blank_table.replace(" ", 0.0)
blank_table = blank_table.set_index(['Sample ID','AnalyteName'])


# Step 2 

### Initialized a new table called step2Table, indexed by Sample ID and Analyte name. The following code converts the columns into necessary datatype, and calculating the standard deviation while appending it to the table


In [244]:
step2Table = pd.DataFrame({"Sample ID"   : rawData['Sample ID'],
                           "AnalyteName" : rawData['Analyte Name'],
                           "Conc (Samp)": rawData['Conc (Samp)'],
                           "SD (Samp)" : rawData['SD (Samp)'],
                          })
step2Table = step2Table.fillna(value = 0.0)
step2Table = step2Table.replace(" ", 0.0)
step2Table = step2Table.set_index(["Sample ID", "AnalyteName"])
step2Table = step2Table.astype({"Conc (Samp)": 'float',
})
step2Table['sum'] = step2Table.sum(axis = 1)
step2Table = step2Table[step2Table['sum'] != 0]
step2Table = step2Table.drop(columns = 'sum')

#Step 2 Table is convered into format of 
#IndexType: Tuple ([Sample ID], [Analyte Name])
#All other columns are in float64 




### Step 3 Rearrange data so Sample and Analyte are on the right 

In [245]:
step3Table = step2Table
#Step3Table is a more precise table that only shows average concentration as well as standard deviation
#Using inspection everything matches up to Sasha's manual calculated result 

### Step 4: Using new naming convention to convert average concentration in sample and std to mg/kg

In [246]:
def blank_matcher():
    """Loop through each row of the table and output a new table after converting average concentration of the element in the sample and the std dev to (mg/kg)
    as well as converting dilution back to original value 
    """
    ans = []
    for info_tuple in step2Table.index:
        dilution_index = info_tuple[0][12]
        salt_index = info_tuple[0][5:7]
        sample_type = info_tuple[0][2]
        if sample_type == 'B':
            ans.append('NA')
        else:
            boo = dilutionTable[dilutionTable['dilution ID'] == info_tuple[0]]
            dilution_constant = (boo['volume digestion (ml)'].unique().astype(float)[0] * boo['dilution factor'].unique().astype(float)[0]) / boo['mass sample (g)'].unique().astype(float)[0]
            blank_name = info_tuple[0][0:2] + 'B' + info_tuple[0][3:]
            calculation = (step2Table.loc[[info_tuple]]['Conc (Samp)'].unique().astype(float)[0] - blank_table.loc[[(blank_name, info_tuple[1])]]['Conc (Calib)'].unique().astype(float)[0]) * dilution_constant
            ans.append(calculation)

    return ans 
a = blank_matcher()
final_table = step2Table
final_table['concentration of element in sample'] = a


In [239]:
#export out for Excel view
final_table.to_csv('output.csv')
